In [1]:
import tensorflow as tf
import datetime
import os
os. environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import *
from tensorflow.keras.metrics import IoU
from tensorflow.keras.layers import Input

from dataset_loader import DataLoader
from model import UNet
from losses import CustomLoss

2025-03-01 15:46:58.312907: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-01 15:46:58.382146: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 15:46:58.832328: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/diazoangga/miniconda3/envs/cvLab/lib/
2025-03-01 15:46:58.832369: W tensorflow/compiler/xla

In [2]:
DATASET = {
    'DATASET_PATH': './Dataset',
    'TRAIN_VAL_RATIO': 0.2,
    'BATCH_SIZE': 4,
    'IMG_SIZE': (128,128),
    'SHUFFLE': True,
    'NUM_CLASSES': 3
    }

AUG = {
    'FLIP_H': 0.3,
    'FLIP_V': 0.6,
    'ROTATE_CW': 0.4,
    'ROTATE_CCW': 0.7,
    'TRANSLATE': [5, 0.3],
    'RAND_BRIGHTNESS': 0.5,
}

RANDOM_SEED = 120

MODEL = {
    'ARCH': 'UNet',
    'INPUT_SIZE': (128,128,3)
    }
LOSS = {
    'DICE_COEFF': 0.1,
    'IOU_COEFF': 0.1,
    'FOCAL_LOSS_COEFF': 0.8
    }

TRAIN = {
    'NUM_EPOCHS': 2,
    
    }


In [3]:
# Define the dataset loader
datasetLoader = DataLoader(DATASET['DATASET_PATH'])
train_dataset, val_dataset, test_dataset = datasetLoader.create_dataset()


Importing the datasets with the following parameters...
   Dataset path                    : ./Dataset
   Train-Val dataset ratio        : 0.2

Splitting training and validation sets...
num of training data:  2944
num of validation data:  736
num of testing data:  3710
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2025-03-01 15:46:59.826563: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-01 15:46:59.830557: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-01 15:46:59.830697: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-01 15:46:59.831703: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-01 15:46:59.831942: I tensorflow/compiler/xla/stream_executo

In [4]:
# Define the model
if MODEL['ARCH'] == 'UNet':
    unet = UNet(num_class=3, dropout=0.2)
    inputs = Input(shape=MODEL['INPUT_SIZE'])
    out = unet(inputs, training=True)
    model = tf.keras.Model(inputs=inputs, outputs=out)
elif MODEL['ARCH'] == 'Autoencoder':
    pass
elif MODEL['ARCH'] == 'CLIP':
    pass
elif MODEL['ARCH'] == 'Prompt':
    pass
else:
    print('Model architecture is not assigned')

In [5]:
# Compile the model
loss = CustomLoss(LOSS)
optimizer = Adam(learning_rate=0.001)
iou_metrics = IoU(3, [0])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

In [6]:
history = model.fit(train_dataset, 
                    validation_data=val_dataset, 
                    epochs=TRAIN['NUM_EPOCHS'], 
                    verbose=1,
                    callbacks=[tensorboard_callback])

Epoch 1/2


2025-03-01 15:47:04.075361: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2025-03-01 15:47:05.723357: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-03-01 15:47:05.723391: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-03-01 15:47:05.765479: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.39GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m

 79/736 [==>...........................] - ETA: 1:07 - loss: 0.9455 - acc: 0.5726

Corrupt JPEG data: 240 extraneous bytes before marker 0xd9


255/736 [=========>....................] - ETA: 49s - loss: 0.8625 - acc: 0.6120

Corrupt JPEG data: premature end of data segment


736/736 [==============================] - 93s 111ms/step - loss: 0.7824 - acc: 0.6607 - val_loss: 0.7444 - val_acc: 0.6687
Epoch 2/2
323/736 [============>.................] - ETA: 40s - loss: 0.7119 - acc: 0.6985

KeyboardInterrupt: 